In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 14 mins
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_l1ai0q
H2O cluster total nodes:,1
H2O cluster free memory:,555 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [4]:
train_high = h2o.import_file('../cleaned-data/04-train_high-catAsInt-withXYZ.csv')
train_low = h2o.import_file('../cleaned-data/04-train_low-catAsInt-withXYZ.csv')
test_high = h2o.import_file('../cleaned-data/04-test_high-catAsInt-withXYZ.csv')
test_low = h2o.import_file('../cleaned-data/04-test_low-catAsInt-withXYZ.csv')      
train_high.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,cut,color,clarity,depth,table,x,y,z,price
8,2.05,4,6,1,62.4,57,8.17,8.09,5.08,18193
34,2.01,0,1,4,64.9,60,7.82,7.78,5.06,13744
42,2,1,2,3,64.2,52,7.91,7.88,5.07,11400
72,2,1,7,3,60.3,56,8.08,8.11,4.88,16425
82,2.28,4,1,4,62.7,57,8.43,8.36,5.26,16450
93,2.02,2,2,1,60.4,58,8.2,8.16,4.94,17357
141,2,3,3,3,60.7,60,8.07,8.11,4.91,15312
154,2.01,3,2,0,61.5,62,8.07,7.94,4.92,15475
165,2.31,3,2,3,62.9,58,8.43,8.34,5.27,17218
167,2.01,4,3,1,60.6,57,8.19,8.09,4.92,17383


In [5]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 11)
(38730, 11)
(521, 10)
(12928, 10)


[None, None, None, None]

In [6]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']

In [7]:
train_frame = train_high
model_name = 'diamonds_high'

aml_high = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |
11:24:10.895: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 4 models.
11:24:21.918: StackedEnsemble_AllModels_AutoML_20200301_112410 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
11:30:06.528: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 7 models.
11:30:17.542: StackedEnsemble_AllModels_AutoML_20200301_113006 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
11:41:32.414: New models will be added to existing leaderboard 

predict
16622.9
16644.4
16417.5
16490.5
15709.3
13944.3
16410.2
15961.6
15505.1
15730.8


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid__1_AutoML_20200301_122417_model_33,2.82789e+06,1681.63,2.82789e+06,1334.92,0.121761
GBM_2_AutoML_20200301_122417,2.82965e+06,1682.15,2.82965e+06,1319.62,0.120664
XGBoost_2_AutoML_20200301_122417,2.83087e+06,1682.52,2.83087e+06,1321.82,0.120537
XGBoost_3_AutoML_20200301_122417,2.83154e+06,1682.72,2.83154e+06,1333.73,0.120722
GBM_4_AutoML_20200301_122417,2.83339e+06,1683.27,2.83339e+06,1318.43,0.120976
StackedEnsemble_BestOfFamily_AutoML_20200301_114132,2.84077e+06,1685.46,2.84077e+06,1325.5,0.120887
XGBoost_grid__1_AutoML_20200301_122417_model_4,2.85738e+06,1690.38,2.85738e+06,1332.93,0.121665
XGBoost_1_AutoML_20200301_122417,2.85918e+06,1690.91,2.85918e+06,1329.84,0.121139
XGBoost_grid__1_AutoML_20200301_122417_model_2,2.86235e+06,1691.85,2.86235e+06,1340.12,0.121972
XGBoost_grid__1_AutoML_20200301_114132_model_2,2.86628e+06,1693.01,2.86628e+06,1344.86,0.121209


In [8]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [9]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |
11:33:24.214: New models will be added to existing leaderboard diamonds_low@@price (leaderboard frame=null) with already 3 models.
11:33:36.232: StackedEnsemble_AllModels_AutoML_20200301_113324 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
11:45:54.283: New models will be added to existing leaderboard diamonds_low@@price (leaderboard frame=null) with already 5 models.
11:50:57.101: StackedEnsemble_AllModels_AutoML_20200301_114554 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
12:29:09.292: New models will be added to existing leaderboard di

predict
4451.18
1041.6
7193.58
1486.48
2206.44
1962.02
1833.01
3446.88
15731.2
3686.82


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20200301_122909,181982,426.594,181982,227.268,0.0915012
XGBoost_2_AutoML_20200301_122909,186692,432.079,186692,226.688,0.0882195
StackedEnsemble_BestOfFamily_AutoML_20200301_114554,208624,456.754,208624,258.348,0.134609
XGBoost_1_AutoML_20200301_114554,210286,458.569,210286,264.219,0.139699
XGBoost_3_AutoML_20200301_122909,231777,481.432,231777,268.757,0.115557
XGBoost_2_AutoML_20200301_114554,242417,492.359,242417,282.272,0.14303
DRF_1_AutoML_20200301_114554,311554,558.17,311554,297.041,0.131118
GBM_1_AutoML_20200301_114554,424522,651.554,424522,382.178,0.200674
XGBoost_3_AutoML_20200301_114554,468477,684.453,468477,378.891,nan
StackedEnsemble_AllModels_AutoML_20200301_113230,514411,717.224,514411,438.472,nan


In [10]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [11]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [14]:
h2o.export_file(frame=combinedTable,path='../predictions/11-Auto-ML-Split-data-catAsInt-withXYZ.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
